In [2]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.9 MB/s 
     |████████████████████████████████| 182 kB 72.1 MB/s 


In [4]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 25.8 MB/s 
     |████████████████████████████████| 132 kB 51.5 MB/s 
     |████████████████████████████████| 212 kB 52.1 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
data = pd.read_csv("/content/drive/MyDrive/train_data.csv", on_bad_lines='skip')

In [6]:
model_name = "roberta-base"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
#preprocessing
def preprocessing(data):
  import nltk
  from nltk.stem.porter import PorterStemmer
  import re
  from nltk.corpus import stopwords
  ps = PorterStemmer()
  sentences = []
  nltk.download("stopwords")

  for i in range(data.shape[0]): # data.shape[0]
    sentence = re.sub('[^a-zA-Z]',' ',data.iloc[i].values[0]) # noktalama işaretlerini silme.
    sentence = sentence.lower() # kelimelerin hepsini küçük harf 
    sentence = sentence.split() # keliemeleri listeye atma
    sentence = [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words("english"))] #anlamı olmayan(stopwords) kelimeleri listeden atma
    sentence = " ".join(sentence) # listede kalan kelimeleri birleştirip yeni cümleyi oluşturma.
    sentences.append(sentence)
  df = pd.DataFrame(data = sentences,columns = ["Haber"])
  df["Truth"] = data["Truth"]
  return df

In [9]:
preprocessing_data = preprocessing(data)
preprocessing_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Haber,Truth
0,undercov video prove trump right voter fraud n...,0
1,club face fa cup first round,1
2,world cup injuri watch doubt,1
3,statu world cup winner charlton unveil,1
4,polic apolog pursu sex assault report,1
...,...,...
42198,turkey close border iraq evacu embassi,1
42199,storm arwen damag tree use help save salmon,1
42200,iraq war veteran take cowardli republican wood...,0
42201,deplor hillari got bust offend half basecal be...,0


In [10]:
def process_data(row):
    sentence = row[0]
    truth_value = row[1]
    sentence = " ".join(sentence.split())
    encodings = tokenizer(sentence,padding = "max_length",truncation =True,max_length = 128)
    encodings["label"] = truth_value
    encodings["text"] = sentence
    return encodings

In [11]:
processed_data = []
for i in range(0,preprocessing_data.shape[0]):
    processed_data.append(process_data(preprocessing_data.iloc[i]))

In [12]:
veri = pd.DataFrame(data = processed_data)
veri

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 5087, 438, 1417, 569, 3364, 20125, 235, 66...",0,undercov video prove trump right voter fraud n...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 21858, 652, 18363, 4946, 78, 1062, 2, 1, 1...",1,club face fa cup first round
2,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 8331, 4946, 30272, 6151, 1183, 2980, 2, 1,...",1,world cup injuri watch doubt
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 24344, 257, 232, 4946, 1924, 16224, 462, 1...",1,statu world cup winner charlton unveil
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 642, 12589, 23842, 44276, 2099, 2080, 266,...",1,polic apolog pursu sex assault report
...,...,...,...,...
42198,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 18346, 5282, 593, 1424, 10209, 18611, 1372...",1,turkey close border iraq evacu embassi
42199,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 10868, 4709, 11760, 9656, 1073, 3907, 304,...",1,storm arwen damag tree use help save salmon
42200,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 3578, 1343, 997, 3142, 185, 35488, 3572, 3...",0,iraq war veteran take cowardli republican wood...
42201,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 2794, 2911, 368, 9910, 1512, 300, 11044, 3...",0,deplor hillari got bust offend half basecal be...


In [13]:
from sklearn.model_selection import train_test_split
train_df,valid_df = train_test_split(veri,test_size = 0.2)

In [14]:
import pyarrow as pa
from datasets import Dataset
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name) #,num_labels=2

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [16]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Colab Notebooks/roberta-base/result",evaluation_strategy="epoch")
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer = tokenizer,
)

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33762
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12663
  Number of trainable parameters = 124647170
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the

Epoch,Training Loss,Validation Loss
1,0.326100,0.248635
2,0.722500,1.475017
3,0.134300,0.137510


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/result/che

TrainOutput(global_step=12663, training_loss=0.3341316930172446, metrics={'train_runtime': 3020.8884, 'train_samples_per_second': 33.529, 'train_steps_per_second': 4.192, 'total_flos': 6662366588298240.0, 'train_loss': 0.3341316930172446, 'epoch': 3.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8441
  Batch size = 8


{'eval_loss': 0.1375100463628769,
 'eval_runtime': 57.2371,
 'eval_samples_per_second': 147.474,
 'eval_steps_per_second': 18.45,
 'epoch': 3.0}

In [19]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/roberta-base/model")

Configuration saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/model/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/roberta-base/model/pytorch_model.bin
